In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("dataUnderstanding").getOrCreate()

medical = spark.read.csv("medical_data.csv", header=True, inferSchema=True)
patient = spark.read.csv("patient_data.csv", header=True, inferSchema=True)

23/10/10 17:24:31 WARN Utils: Your hostname, Stephens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
23/10/10 17:24:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 17:24:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/10 17:24:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/10 17:24:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/10/10 17:24:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
medical = medical.withColumn("chol", medical.chol.cast('integer'))
medical = medical.withColumn("exang", medical.exang.cast('integer'))
medical.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



In [3]:
patient.printSchema()
patient = patient.withColumn("sex", patient.sex.cast("integer"))
patient.printSchema()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)



In [4]:
print(spark.createDataFrame(medical.describe().toPandas().transpose().reset_index()).show(truncate=False))

23/10/10 17:24:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----+-------------------+-------------------+---+----+
|index   |0    |1                  |2                  |3  |4   |
+--------+-----+-------------------+-------------------+---+----+
|summary |count|mean               |stddev             |min|max |
|id      |1025 |513.0              |296.0363153398583  |1  |1025|
|cp      |1025 |0.9424390243902439 |1.029640743645865  |0  |3   |
|trestbps|1025 |131.61170731707318 |17.516718005376408 |94 |200 |
|chol    |982  |246.01221995926682 |51.7080514274769   |126|564 |
|fbs     |1025 |0.14926829268292682|0.35652668972715756|0  |1   |
|restecg |1025 |0.5297560975609756 |0.5278775668748918 |0  |2   |
|thalach |1025 |149.11414634146342 |23.00572374597721  |71 |202 |
|exang   |965  |0.3378238341968912 |0.4732134233484412 |0  |1   |
|oldpeak |1025 |1.0715121951219524 |1.1750532551501767 |0.0|6.2 |
|slope   |1025 |1.3853658536585365 |0.6177552671745918 |0  |2   |
|ca      |1025 |0.7541463414634146 |1.0307976650242825 |0  |4   |
|thal    |

In [5]:
print(spark.createDataFrame(patient.describe().toPandas().transpose().reset_index()).show(truncate=False))


+-------+-----+------------------+-------------------+---+----+
|index  |0    |1                 |2                  |3  |4   |
+-------+-----+------------------+-------------------+---+----+
|summary|count|mean              |stddev             |min|max |
|id     |1025 |513.0             |296.0363153398583  |1  |1025|
|age    |1025 |54.43414634146342 |9.072290233244278  |29 |77  |
|sex    |987  |0.7001013171225937|0.45844563856310544|0  |1   |
+-------+-----+------------------+-------------------+---+----+

None


In [6]:
from pyspark.sql.functions import col

# Check and print missing values in the patient DataFrame
print("Missing values in the patient DataFrame:")
for col_name in patient.columns:
    missing_count = patient.filter((col(col_name) == "NA") | (col(col_name).isNull())).count()
    print(f"{col_name}: {missing_count}")

print()

# Check and print missing values in the medical DataFrame
print("Missing values in the medical DataFrame:")
for col_name in medical.columns:
    missing_count = medical.filter((col(col_name) == "NA") | (col(col_name).isNull())).count()
    print(f"{col_name}: {missing_count}")

Missing values in the patient DataFrame:
id: 0
age: 0
sex: 38

Missing values in the medical DataFrame:
id: 0
cp: 0
trestbps: 0
chol: 43
fbs: 0
restecg: 0
thalach: 0
exang: 60
oldpeak: 0
slope: 0
ca: 0
thal: 0
target: 0


In [7]:
medical_no_id = medical.drop("id")
medical_no_id.show()

patient_no_id = patient.drop("id")
patient_no_id.show()

+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|   3|     0|
|  0|     140| 203|  1|      0|    155|    1|    3.1|    0|  0|   3|     0|
|  0|     145| 174|  0|      1|    125|    1|    2.6|    0|  0|   3|     0|
|  0|     148| 203|  0|      1|    161|    0|    0.0|    2|  1|   3|     0|
|  0|     138| 294|  1|      1|    106|    0|    1.9|    1|  3|   2|     0|
|  0|     100| 248|  0|      0|    122|    0|    1.0|    1|  0|   2|     1|
|  0|     114| 318|  0|      2|    140|    0|    4.4|    0|  3|   1|     0|
|  0|     160| 289|  0|      0|    145|    1|    0.8|    1|  1|   3|     0|
|  0|     120| 249|  0|      0|    144|    0|    0.8|    2|  0|   3|     0|
|  0|     122| 286|  0|      0|    116|    1|    3.2|    1|  2|   2|     0|
|  0|     11

In [8]:
from ydata_profiling import ProfileReport
patient_profile = ProfileReport(patient_no_id.toPandas(), title="Patient Data")
medical_profile = ProfileReport(medical_no_id.toPandas(), title="Medical Data")

In [9]:

medical_profile.to_file("medical_profile.html")
patient_profile.to_file("patient_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

23/10/11 03:19:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1023361 ms exceeds timeout 120000 ms
23/10/11 03:19:51 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/11 03:19:54 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 